# Electoral College Data Analysis

The purpose of this project is to analyze the how the electoral college votes were distributed after the 2010 census, look at how the redistribution of population (by estimate) has shifted over time until today, and what that means for the voter per electoral college vote in each state. I will also analyze how we expect the electoral college to be redistributed after the 2020 census, given census bureau predictions.

An analysis will also be performed on the percentage likelihood of each state giving its electoral college votes to a particular party and their respective nominees, based purely on historical data. The aim is to demonstrate which states have the greatest power per vote, given both their current electoral votes allotted and the likelihood of that state assigning thier votes to either candidate.

### Vizualizations
- viz of state pops over time (5 lowest, 5 highest)
- animation of voter potency (voters / electoral vote)
- electoral votes w/ new census
- voter potency w/ new census
- guaranteed (statistically) electoral votes for each party
- swing states and leanings and votes

## Pull Data for Analysis

In [ ]:
conn = sqlite3.connect('census_pop_data.db')
c = conn.cursor()

In [ ]:
# init dictionary for states and their populations, house seats, and priority value
state_dict = {}

# query database and put data to manipulate in dictionary
c.execute('SELECT state, (pop + overseas_pop) as total_pop \
           FROM population \
           WHERE year = 2010 AND state NOT IN ("District of Columbia", "Puerto Rico") \
           ORDER BY state')
for state in c.fetchall():
    state_dict.update({state[0]: {'Population': state[1],
                                  'House_seats': 1,
                                  'Priority_value': priority_value(state[1], 2)}})
    
display(state_dict)
display(len(state_dict.keys()))

In [ ]:
def distribute_house_seats(state_dict):
    """
    Iterates through all available house seats after initial 50 are distributed (one to each state)
    to determine who should get each next house seat according to priority value
    """
        
    distributed_dict = copy.deepcopy(state_dict)
    #iterates over all available house seats, range chosen for actual house seat being evaluated
    #whichever state is highest, add one house seat and recalculate the priority value
    for i in range(51, 436):
        next_state = assign_next_seat(distributed_dict)
        distributed_dict[next_state]['House_seats'] += 1
        distributed_dict[next_state]['Priority_value'] = priority_value(distributed_dict[next_state]['Population'], 
                                                                        distributed_dict[next_state]['House_seats']+1)
    
    #return the dictionary with properly distributed house seats    
    return distributed_dict

In [ ]:
def assign_next_seat(distributed_dict):
    """
    Assigns the next house seat available according to current priority values for each state
    Returns a list with the first value as the state who should get the next seat
    """
    top_value = ["",0]
    
    #iterate through the dictionary keys and keep the state and value for whoever has the 
    #highest priority value
    for state in distributed_dict.keys():
        if distributed_dict[state]['Priority_value'] > top_value[1]:
            top_value = [state, distributed_dict[state]['Priority_value']]  
    
    #return just the state with the top priority value
    return top_value[0]

In [ ]:
# creates a dictionary with house seats properly distributed
apportioned_dict = distribute_house_seats(state_dict)

In [ ]:
# reads census apportionment of house seats into pandas dataframe and cleans
census_apportionment = pd.read_excel('ApportionmentPopulation2010.xls', skiprows=10, usecols=[0,3])[:50]
census_apportionment.columns = ['State','House_seats']
census_apportionment['House_seats'] = census_apportionment.House_seats.astype('int64')
census_apportionment.set_index('State', inplace=True)

display(census_apportionment.head())
display(census_apportionment.info())

In [ ]:
# prints out the state, the calculated house seats through my code, the actual assigned seats according
# to the census bureau, and if they match (to check that my math matches theirs)

print('State', '\t\t', 'Calc', '\t', 'Actual', 'Match')
print()
for state in apportioned_dict.keys():
    if len(state) <= 6:
        print(state, '\t\t', apportioned_dict[state]['House_seats'], '\t', census_apportionment.loc[state][0], 
              '\t', apportioned_dict[state]['House_seats'] == census_apportionment.loc[state][0])
    else:
        print(state, '\t', apportioned_dict[state]['House_seats'], '\t', census_apportionment.loc[state][0], 
              '\t', apportioned_dict[state]['House_seats'] == census_apportionment.loc[state][0]) 